In [1]:
import pandas as pd
import numpy as np
import os
import requests
import json
from datetime import datetime


In [2]:
# Create a list with all files in the folder using a list comprehension

folderpath = r"/Users/ru/City-Bike_Analysis/2022-citibike-tripdata" # make sure to put the 'r' in front
filepath = [os.path.join(folderpath, name) for name in os.listdir(folderpath)]

In [3]:
filepath

['/Users/ru/City-Bike_Analysis/2022-citibike-tripdata/202209-citibike-tripdata.zip',
 '/Users/ru/City-Bike_Analysis/2022-citibike-tripdata/202201-citibike-tripdata.zip',
 '/Users/ru/City-Bike_Analysis/2022-citibike-tripdata/.DS_Store',
 '/Users/ru/City-Bike_Analysis/2022-citibike-tripdata/202203-citibike-tripdata.zip',
 '/Users/ru/City-Bike_Analysis/2022-citibike-tripdata/202205-citibike-tripdata.zip',
 '/Users/ru/City-Bike_Analysis/2022-citibike-tripdata/202207-citibike-tripdata.zip',
 '/Users/ru/City-Bike_Analysis/2022-citibike-tripdata/202211-citibike-tripdata.zip',
 '/Users/ru/City-Bike_Analysis/2022-citibike-tripdata/202201-citibike-tripdata 2',
 '/Users/ru/City-Bike_Analysis/2022-citibike-tripdata/202208-citibike-tripdata.zip',
 '/Users/ru/City-Bike_Analysis/2022-citibike-tripdata/202202-citibike-tripdata.zip',
 '/Users/ru/City-Bike_Analysis/2022-citibike-tripdata/202212-citibike-tripdata.zip',
 '/Users/ru/City-Bike_Analysis/2022-citibike-tripdata/202204-citibike-tripdata.zip',
 

I got quite a lot of RAM problems and needed to work around them, this is wy the following code does not follow the proposed code in the exercise

In [4]:
# import of zipfiles and reducing RAM proböems throuch chunks and parquet (proposed by chat gpt)
import zipfile

In [5]:
from pathlib import Path

In [6]:
import gc

In [7]:
chunk_size = 100_000
filepath = list(Path("/Users/ru/City-Bike_Analysis/2022-citibike-tripdata").glob("*.zip"))
output_folder = Path("chunks_parquet")
output_folder.mkdir(exist_ok=True)

# Spaltentypen vorab definieren, um DtypeWarnings zu vermeiden
dtype_dict = {
    'start_station_id': str,
    'end_station_id': str,
    'bike_id': str
}

for zip_file in filepath:
    print(f"Verarbeite: {zip_file}")
    with zipfile.ZipFile(zip_file) as z:
        for name in z.namelist():
            if name.endswith(".csv"):
                print(f"  CSV: {name}")
                # low_memory=False für konsistente Typen
                for i, chunk in enumerate(pd.read_csv(z.open(name), chunksize=chunk_size, dtype=dtype_dict, low_memory=False)):
                    
                    # Datumsspalte vorbereiten
                    if 'started_at' in chunk.columns:
                        chunk['started_at'] = pd.to_datetime(chunk['started_at'], errors='coerce')
                        chunk['date'] = chunk['started_at'].dt.floor('D')

                    # IDs als Strings (NaN → '')
                    id_cols = ['start_station_id', 'end_station_id', 'bike_id']
                    for col in id_cols:
                        if col in chunk.columns:
                            chunk[col] = chunk[col].fillna('')

                    # Direkt als Parquet speichern
                    out_path = output_folder / f"{zip_file.stem}_{name}_{i}.parquet"
                    chunk.to_parquet(out_path, index=False)

                    # Chunk aus RAM löschen, Garbage Collector aufrufen
                    del chunk
                    gc.collect()

                print(f"    Fertig: {name}")
    print(f"  Fertig: {zip_file}")

Verarbeite: /Users/ru/City-Bike_Analysis/2022-citibike-tripdata/202209-citibike-tripdata.zip
  CSV: 202209-citibike-tripdata_4.csv
    Fertig: 202209-citibike-tripdata_4.csv
  CSV: 202209-citibike-tripdata_1.csv
    Fertig: 202209-citibike-tripdata_1.csv
  CSV: 202209-citibike-tripdata_2.csv
    Fertig: 202209-citibike-tripdata_2.csv
  CSV: 202209-citibike-tripdata_3.csv
    Fertig: 202209-citibike-tripdata_3.csv
  Fertig: /Users/ru/City-Bike_Analysis/2022-citibike-tripdata/202209-citibike-tripdata.zip
Verarbeite: /Users/ru/City-Bike_Analysis/2022-citibike-tripdata/202201-citibike-tripdata.zip
  CSV: 202201-citibike-tripdata_2.csv
    Fertig: 202201-citibike-tripdata_2.csv
  CSV: 202201-citibike-tripdata_1.csv
    Fertig: 202201-citibike-tripdata_1.csv
  Fertig: /Users/ru/City-Bike_Analysis/2022-citibike-tripdata/202201-citibike-tripdata.zip
Verarbeite: /Users/ru/City-Bike_Analysis/2022-citibike-tripdata/202203-citibike-tripdata.zip
  CSV: 202203-citibike-tripdata_2.csv
    Fertig: 202

In [8]:
# create a ful list again
import glob
parquet_files = glob.glob("chunks_parquet/*.parquet")
dfs = [pd.read_parquet(f) for f in parquet_files]
df = pd.concat(dfs, ignore_index=True)

In [9]:
df.shape

(29838806, 14)

In [10]:
df.head()


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,date
0,48D031922380C891,classic_bike,2022-06-30 14:48:29.106,2022-06-30 14:52:36.060,E 2 St & 2 Ave,5593.02,E 4 St & 2 Ave,5593.04,40.725029,-73.990697,40.726281,-73.989780,casual,2022-06-30
1,626ED023AB7B878B,classic_bike,2022-06-11 23:24:42.852,2022-06-11 23:32:35.683,4 Ave & 9 St,3955.05,S Portland Ave & Hanson Pl,4354.05,40.670513,-73.988766,40.685396,-73.974315,member,2022-06-11
2,F34E7414A2296D1E,electric_bike,2022-06-16 18:32:32.755,2022-06-16 18:49:22.428,Henry St & Atlantic Ave,4531.05,Cambridge Pl & Gates Ave,4377.04,40.690892,-73.996138,40.684880,-73.963040,member,2022-06-16
3,065A8CA580FC546C,classic_bike,2022-06-12 17:55:42.712,2022-06-12 18:03:23.733,W 13 St & 5 Ave,5947.04,9 Ave & W 18 St,6190.08,40.735445,-73.994310,40.743174,-74.003664,member,2022-06-12
4,84EB03D3B39A9BE2,classic_bike,2022-06-08 17:47:41.306,2022-06-08 17:55:30.131,6 Ave & Canal St,5500.07,W 18 St & 6 Ave,6064.08,40.722438,-74.005664,40.739713,-73.994564,member,2022-06-08


In [11]:
df.tail()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,date
29838801,E634864F6FA99245,classic_bike,2022-08-12 20:09:55.511,2022-08-12 20:17:18.983,W 36 St & 7 Ave,6483.06,W 35 St & Dyer Ave,6569.08,40.752149,-73.989539,40.754692,-73.997402,member,2022-08-12
29838802,02BE424C598155F7,electric_bike,2022-08-29 14:34:00.870,2022-08-29 14:39:45.351,Henry St & Degraw St,4380.08,Smith St & 9 St,4077.04,40.684662,-73.999127,40.674696,-73.997858,member,2022-08-29
29838803,33D66AE4BCA0286C,electric_bike,2022-08-13 18:35:23.911,2022-08-13 18:41:45.149,3 St & 3 Ave,4028.03,Smith St & 9 St,4077.04,40.675070,-73.987752,40.674696,-73.997858,member,2022-08-13
29838804,FEFFA7D1F437337F,classic_bike,2022-08-24 14:39:31.468,2022-08-24 14:49:41.734,W 36 St & 7 Ave,6483.06,W 54 St & 11 Ave,6955.05,40.752149,-73.989539,40.768333,-73.992573,member,2022-08-24
29838805,C1A58D5156EE725E,classic_bike,2022-08-29 18:34:40.753,2022-08-29 18:56:00.791,5 Ave & E 87 St,7323.09,W 111 St & 5 Ave,7587.16,40.782323,-73.959875,40.797521,-73.948940,casual,2022-08-29


In [12]:
# define token for NOAA 
Token = "xsUjVeQyODAucSZouRAbpkNiHoCoGMuw"


In [13]:
# Get the API 
r = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TAVG&limit=1000&stationid=GHCND:USW00014732&startdate=2022-01-01&enddate=2022-12-31', headers={'token':Token})

In [14]:
# Load the api response as a json
d = json.loads(r.text)

In [15]:
d

{'metadata': {'resultset': {'offset': 1, 'count': 365, 'limit': 1000}},
 'results': [{'date': '2022-01-01T00:00:00',
   'datatype': 'TAVG',
   'station': 'GHCND:USW00014732',
   'attributes': 'H,,S,',
   'value': 116},
  {'date': '2022-01-02T00:00:00',
   'datatype': 'TAVG',
   'station': 'GHCND:USW00014732',
   'attributes': 'H,,S,',
   'value': 114},
  {'date': '2022-01-03T00:00:00',
   'datatype': 'TAVG',
   'station': 'GHCND:USW00014732',
   'attributes': 'H,,S,',
   'value': 14},
  {'date': '2022-01-04T00:00:00',
   'datatype': 'TAVG',
   'station': 'GHCND:USW00014732',
   'attributes': 'H,,S,',
   'value': -27},
  {'date': '2022-01-05T00:00:00',
   'datatype': 'TAVG',
   'station': 'GHCND:USW00014732',
   'attributes': 'H,,S,',
   'value': 32},
  {'date': '2022-01-06T00:00:00',
   'datatype': 'TAVG',
   'station': 'GHCND:USW00014732',
   'attributes': 'H,,S,',
   'value': 49},
  {'date': '2022-01-07T00:00:00',
   'datatype': 'TAVG',
   'station': 'GHCND:USW00014732',
   'attribut

In [16]:
# Secure all items in the response that correspond to TAVG

avg_temps = [item for item in d['results'] if item['datatype']=='TAVG']

In [17]:
# Get only the date field from all average temperature readings

dates_temp = [item['date'] for item in avg_temps]

In [18]:
# Get the temperature from all average temperature readings

temps = [item['value'] for item in avg_temps]

In [20]:
temps

[116,
 114,
 14,
 -27,
 32,
 49,
 7,
 -25,
 14,
 16,
 -54,
 -19,
 40,
 48,
 -67,
 -80,
 39,
 18,
 32,
 51,
 -60,
 -59,
 -7,
 -2,
 36,
 -23,
 -42,
 1,
 -48,
 -71,
 -34,
 -17,
 23,
 64,
 58,
 -28,
 -48,
 5,
 41,
 28,
 63,
 86,
 118,
 28,
 -43,
 -47,
 16,
 116,
 99,
 1,
 -26,
 41,
 56,
 144,
 11,
 8,
 -11,
 24,
 8,
 26,
 77,
 56,
 -19,
 31,
 96,
 182,
 87,
 32,
 40,
 72,
 51,
 -23,
 53,
 116,
 136,
 82,
 139,
 131,
 143,
 104,
 107,
 72,
 57,
 96,
 104,
 59,
 -15,
 -12,
 26,
 99,
 131,
 72,
 70,
 76,
 97,
 86,
 91,
 128,
 114,
 86,
 88,
 134,
 154,
 163,
 157,
 157,
 83,
 79,
 76,
 94,
 106,
 146,
 133,
 128,
 104,
 115,
 116,
 82,
 104,
 129,
 136,
 121,
 124,
 120,
 172,
 148,
 100,
 104,
 135,
 154,
 161,
 171,
 160,
 179,
 188,
 204,
 197,
 178,
 155,
 154,
 212,
 278,
 210,
 174,
 162,
 164,
 202,
 204,
 207,
 233,
 283,
 178,
 203,
 195,
 219,
 200,
 222,
 218,
 237,
 244,
 218,
 218,
 205,
 242,
 239,
 233,
 203,
 252,
 217,
 173,
 205,
 221,
 191,
 192,
 219,
 258,
 270,
 243,
 22

In [21]:
# Put the results in a dataframe

df_temp = pd.DataFrame()

In [22]:
# Get only date and cast it to date time; convert temperature from tenths of Celsius to normal Celsius

df_temp['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in dates_temp]
df_temp['avgTemp'] = [float(v)/10.0 for v in temps]

In [23]:
type(df_temp['date'].iloc[0])

pandas._libs.tslibs.timestamps.Timestamp

In [24]:
df_temp.tail()

,date,avgTemp
360,2022-12-27,-0.7
361,2022-12-28,3.4
362,2022-12-29,6.4
363,2022-12-30,9.3
364,2022-12-31,8.2


In [25]:
df_temp.head()

,date,avgTemp
0,2022-01-01,11.6
1,2022-01-02,11.4
2,2022-01-03,1.4
3,2022-01-04,-2.7
4,2022-01-05,3.2


In [26]:
# check datatypes of citi bike data
df.dtypes

ride_id                       object
rideable_type                 object
started_at            datetime64[ns]
ended_at                      object
start_station_name            object
start_station_id              object
end_station_name              object
end_station_id                object
start_lat                    float64
start_lng                    float64
end_lat                      float64
end_lng                      float64
member_casual                 object
date                  datetime64[ns]
dtype: object

In [27]:
df.head(5)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,date
0,48D031922380C891,classic_bike,2022-06-30 14:48:29.106,2022-06-30 14:52:36.060,E 2 St & 2 Ave,5593.02,E 4 St & 2 Ave,5593.04,40.725029,-73.990697,40.726281,-73.989780,casual,2022-06-30
1,626ED023AB7B878B,classic_bike,2022-06-11 23:24:42.852,2022-06-11 23:32:35.683,4 Ave & 9 St,3955.05,S Portland Ave & Hanson Pl,4354.05,40.670513,-73.988766,40.685396,-73.974315,member,2022-06-11
2,F34E7414A2296D1E,electric_bike,2022-06-16 18:32:32.755,2022-06-16 18:49:22.428,Henry St & Atlantic Ave,4531.05,Cambridge Pl & Gates Ave,4377.04,40.690892,-73.996138,40.684880,-73.963040,member,2022-06-16
3,065A8CA580FC546C,classic_bike,2022-06-12 17:55:42.712,2022-06-12 18:03:23.733,W 13 St & 5 Ave,5947.04,9 Ave & W 18 St,6190.08,40.735445,-73.994310,40.743174,-74.003664,member,2022-06-12
4,84EB03D3B39A9BE2,classic_bike,2022-06-08 17:47:41.306,2022-06-08 17:55:30.131,6 Ave & Canal St,5500.07,W 18 St & 6 Ave,6064.08,40.722438,-74.005664,40.739713,-73.994564,member,2022-06-08


In [28]:
# prepare for merging: create common key with the date
df['started_at'] = pd.to_datetime(df['started_at'])

In [29]:
df['date'] = df['started_at'].dt.floor('D')

In [30]:
# merging with chunks
import gc

In [35]:
output_folder = Path("merged_chunks")
output_folder.mkdir(exist_ok=True)



Becuase of Problems with size and RAM I decided to use a sample. 
this is what caht gpt proposed with the following explanation: “All spatial and temporal analyses are based on a 1 % stratified sample by month and user type. This approach preserves seasonal and behavioral patterns while ensuring computational efficiency.”

In [36]:
chunk_size = 100_000

for i in range(0, len(df), chunk_size):
    chunk = df.iloc[i:i+chunk_size].copy()

    chunk_merged = chunk.merge(df_temp, how="left", on="date")

    # Hilfsspalte
    chunk_merged["year_month"] = chunk_merged["started_at"].dt.to_period("M")

    # Stratified Sampling
    chunk_sample = (
        chunk_merged
        .groupby(
            ["year_month", "member_casual"],
            group_keys=False
        )
        .apply(
            lambda x: x.sample(
                frac=0.01 if len(x) > 100 else 1.0,
                random_state=42
            ),
            include_groups=False
        )
    )

    # save sample as csv
      
    chunk_sample.to_csv(
        "citibike_sample_1pct.csv",
        mode="a",
        header=(i == 0),
        index=False
    )
    
 # RAM freigeben
    del chunk, chunk_merged, chunk_sample
    gc.collect()